In [1]:
!pip install -r local_req.txt

  Obtaining dependency information for numpy==1.25.2 from https://files.pythonhosted.org/packages/72/b2/02770e60c4e2f7e158d923ab0dea4e9f146a2dbf267fec6d8dc61d475689/numpy-1.25.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for boto3==1.28.22 from https://files.pythonhosted.org/packages/2f/83/07bbd31d54f8cfbb30cd09802c2eced29a50324f746520eefd69cc7002f6/boto3-1.28.22-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.32.0,>=1.31.22 from https://files.pythonhosted.org/packages/d4/4a/dcec308c412000a3a141be50568dd96b86cbd829ac9ee0b937b4fa6c56ae/botocore-1.31.23-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.3/15.5 MB 6.5 MB/s eta 0:00:03
   - -------------------------------------- 0.7/15.5 MB 9.2 MB/s eta 0:00:02
   -- ------------------------------------- 1.2/15.5 MB 8.2 MB/s eta 0:00:02
   -- ------------------------------------- 1.2/15.5 MB 8.2 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.31.23 which is incompatible.
awscli 1.29.21 requires botocore==1.31.21, but you have botocore 1.31.23 which is incompatible.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.25.2 which is incompatible.


In [2]:
import torch
import torchaudio
import json

In [11]:
def pre_process(audio_path):
    waveform, sr = torchaudio.load(audio_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    return waveform

In [12]:
def get_label(label_pred):
    if type(label_pred) == list:
        label_pred = label_pred[0]
    if 'json_dict' not in globals():
        global json_dict
        with open("labels.json", "r") as f:
            json_dict = json.load(f)
    label = json_dict[str(label_pred)]
    return label

In [14]:
from BEATs import BEATs, BEATsConfig

def predict(audio_path, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    cfg = BEATsConfig(checkpoint['cfg'])
    BEATs_model = BEATs(cfg)
    BEATs_model.load_state_dict(checkpoint['model'])
    BEATs_model.eval()

    waveform = pre_process(audio_path)
    probs = BEATs_model.extract_features(waveform, padding_mask=None)[0]
    return probs

In [25]:
probs = predict('baby.wav', 'model.pt')
label_pred = probs.topk(k=1)[1].tolist()[0][0]
label = get_label(label_pred)
print(probs.size())
print(label_pred)
print(label)

torch.Size([1, 527])
431
Wail, moan
